In [37]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
pd.options.display.float_format = '{:,.3f}'.format

pd.options.mode.chained_assignment = None
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")
#sns.set(rc={'figure.figsize':(10,10)})
import matplotlib
#matplotlib.use('agg')
#matplotlib.style.use('ggplot')
import pickle as pkl
from matplotlib import pyplot as plt
#plt.rcParams.update(plt.rcParamsDefault)
new_plot_col=list(range(1900,2001,10))
from scipy.stats.stats import pearsonr

In [38]:
new_plot_col

[1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000]

In [9]:
def lemma_maker(x, y):
    return lemmatizer.lemmatize(x,y)
br_to_us=pd.read_excel("Book.xlsx",skiprows=[0])
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))
spelling_replacement={'modifier':br_to_us_dict,'head':br_to_us_dict}

In [2]:
contextwords_df=pd.read_csv("contexts.csv",sep="\t")
contextwords=contextwords_df.context.tolist()

In [12]:
reddy11_study=pd.read_csv("/data/dharp/compounding/datasets/ijcnlp_compositionality_data/MeanAndDeviations.clean.txt",sep="\t")
#print(reddy11_study.columns)
reddy11_study.columns=['compound','to_divide']
reddy11_study['modifier_mean'],reddy11_study['modifier_std'],reddy11_study['head_mean'],reddy11_study['head_std'],reddy11_study['compound_mean'],reddy11_study['compound_std'],_=reddy11_study.to_divide.str.split(" ",7).str
reddy11_study['modifier'],reddy11_study['head']=reddy11_study['compound'].str.split(" ",2).str
reddy11_study.modifier=reddy11_study.modifier.str[:-2]
reddy11_study['head']=reddy11_study['head'].str[:-2]
reddy11_study.drop(['compound','to_divide'],axis=1,inplace=True)
#reddy11_study['modifier']=np.vectorize(lemma_maker)(reddy11_study['modifier'],'n')
reddy11_study['head']=np.vectorize(lemma_maker)(reddy11_study['head'],'n')
reddy11_study.replace(spelling_replacement,inplace=True)
reddy11_study['modifier']=reddy11_study['modifier']+"_n"
reddy11_study['head']=reddy11_study['head']+"_n"
reddy11_study=reddy11_study.apply(pd.to_numeric, errors='ignore')
#reddy11_study.set_index(['modifier','head'],inplace=True)
reddy11_study.head()

,modifier_mean,modifier_std,head_mean,head_std,compound_mean,compound_std,modifier,head
0,3.867,1.118,4.867,0.340,4.250,0.871,end_n,user_n
1,1.607,1.655,1.893,1.496,1.704,1.717,firing_n,line_n
2,2.821,1.965,4.862,0.345,3.828,1.234,game_n,plan_n
3,4.767,0.423,4.862,0.345,4.800,0.476,application_n,form_n
4,0.600,0.800,4.586,1.099,1.310,1.021,snail_n,mail_n


In [3]:
compounds=pd.read_csv("/data/dharp/compounding/datasets/compounds.csv")
#compounds=compounds.query('decade != 2000')
#compounds=compounds.reindex()
compounds=compounds.groupby(['modifier','head','decade'])['count'].sum().to_frame()
compounds.info()
compounds.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9078867 entries, (a_n, aaaaa_n, 1870) to (zzzz_n, best_n, 2000)
Data columns (total 1 columns):
count    int64
dtypes: int64(1)
memory usage: 148.7+ MB


count
modifier head    decade       
a_n      aaaaa_n 1870        1
                 1910        1
                 1920        3
                 1930        1
                 1940        1

In [42]:
compounds_new=compounds[compounds['count']>20]
print((compounds.shape[0]-compounds_new.shape[0])/compounds.shape[0])
compounds_new.head()

0.5122525751285926


count
modifier head      decade       
a_n      addison_n 1960       28
                   1970       96
                   1980      206
                   1990      196
                   2000      222

## Before

In [43]:
count_df=reddy11_study[['modifier','head']].merge(compounds_new.reset_index(),on=['modifier','head'],how='left')
#temp_df.set_index(['modifier','head','decade'],inplace=True)
count_df=pd.pivot_table(count_df,index=['modifier','head'],columns='decade',values='count',aggfunc=np.sum)
to_change=[str(x).split('.')[0] for x in count_df.columns.tolist()]
count_df.columns=to_change
count_df.columns = count_df.columns.map(int)
count_df.fillna(0,inplace=True)

In [48]:
selected_df=count_df[new_plot_col]
selected_df[(selected_df != 0).all(1)]

,,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000
modifier,head,,,,,,,,,,,
application_n,form_n,262.000,478.000,571.000,821.000,"2,032.000","4,118.000","7,143.000","13,208.000","21,305.000","39,508.000","44,324.000"
balance_n,sheet_n,"19,934.000","43,172.000","78,268.000","76,070.000","77,632.000","94,371.000","136,029.000","172,020.000","281,296.000","447,042.000","659,867.000"
bank_n,account_n,"2,747.000","4,564.000","4,115.000","5,132.000","5,309.000","8,446.000","14,357.000","24,595.000","42,396.000","80,839.000","149,222.000"
brass_n,ring_n,303.000,251.000,173.000,132.000,149.000,152.000,247.000,416.000,778.000,"1,581.000","2,647.000"
brick_n,wall_n,"8,667.000","6,768.000","5,567.000","5,112.000","4,975.000","7,087.000","10,194.000","13,267.000","19,520.000","32,990.000","60,194.000"
case_n,study_n,59.000,264.000,"2,243.000","9,568.000","11,474.000","29,932.000","79,411.000","216,616.000","390,239.000","737,327.000","1,258,029.000"
chain_n,reaction_n,25.000,69.000,78.000,657.000,"7,060.000","21,994.000","35,146.000","35,001.000","42,120.000","93,356.000","114,377.000"
china_n,clay_n,565.000,322.000,358.000,274.000,300.000,381.000,408.000,578.000,523.000,415.000,598.000
crocodile_n,tear_n,72.000,71.000,204.000,136.000,221.000,257.000,464.000,548.000,487.000,512.000,869.000


In [9]:
compound_counts[compound_counts['count']>20].head(20)

count
modifier head      decade       
a_n      addison_n 1960       28
                   1970       96
                   1980      206
                   1990      196
                   2000      222
         anti_n    1960       27
                   1970       30
                   1980       41
                   1990       67
                   2000       90
         archive_n 1960       98
                   1970      258
                   1980       91
                   1990       60
                   2000       29
         atom_n    1950       29
                   1960       34
                   2000       25
         b_n       1810       58
                   1820       74

In [ ]:
compound_counts[compound_counts['count']>20]

In [14]:
compounds.reset_index(inplace=True)
new_compounds=compounds.loc[compounds.modifier.isin(contextwords) & (compounds['head'].isin(contextwords))]
new_compounds_decade_counts=new_compounds.groupby(['modifier','head'])['count'].sum().to_frame()
new_compounds_decade_counts

count
modifier head                 
a_n      addison_n         526
         anti_n            169
         archive_n         510
         atom_n            116
         b_n             24600
         ballet_n           85
         band_n            384
         baseball_n         90
         behavior_n       1080
         bell_n            665
         belle_n            78
         bend_n            304
         berkley_n         144
         bibliography_n   2799
         bill_n            336
         bird_n            215
         board_n            80
         book_n             37
         boswell_n         214
         brand_n            48
         bunch_n           514
         burn_n            104
         business_n         70
         c_n               756
         calendar_n         78
         call_n             39
         campaign_n        240
         candle_n           48
         cassell_n         510
         cat_n             388
...                        ...
zurich_n ch_n               54
         chamber_n         283
         church_n           28
         city_n             53
         cohort_n          169
         conference_n      300
         congress_n         47
         dada_n            102
         federal_n          42
         government_n      245
         institute_n        98
         james_n           146
         market_n           53
         medical_n         211
         opera_n            44
         peasantry_n       141
         polytechnic_n     363
         school_n          432
         seminar_n        1253
         speech_n           43
         stock_n           431
         translation_n      39
         university_n      371
         wilhelm_n          48
zvi_n    bodie_n            97
         institute_n       918
         press_n            28
         publication_n      38
zygote_n nucleus_n          42
         result_n           72

[848435 rows x 1 columns]

In [15]:
new_compounds_decade_counts.describe()

,count
count,848435.000
mean,2673.265
std,148101.622
min,1.000
25%,74.000
50%,184.000
75%,554.000
max,118319029.000
